In [6]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential,Model,initializers,layers,Input
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [7]:
def Stem(x):
    x=layers.Conv2D(filters=32,kernel_size=(3,3),strides=(2,2),padding='same',activation='relu')(x)
    x=layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
    x=layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)

    x1=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
    x2=layers.Conv2D(filters=96,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
    x3=tf.keras.layers.Concatenate()([x1, x2])

    x4=layers.Conv2D(filters=64,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x3)
    x4=layers.Conv2D(filters=96,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x4)

    x5=layers.Conv2D(filters=64,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x3)
    x5=layers.Conv2D(filters=64,kernel_size=(7,1),strides=(1,1),padding='same',activation='relu')(x5)
    x5=layers.Conv2D(filters=64,kernel_size=(1,7),strides=(1,1),padding='same',activation='relu')(x5)
    x5=layers.Conv2D(filters=96,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x5)
    x6=tf.keras.layers.Concatenate()([x4, x5])

    x7=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x6)
    x8=layers.Conv2D(filters=192,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x6)
    x9=tf.keras.layers.Concatenate()([x7, x8])

    return x9


def Inception_ResNet_Block_A(x):
    x=layers.activation.ReLU()(x)

    x1=layers.Conv2D(filters=32,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)

    x2=layers.Conv2D(filters=32,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x2=layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x2)


    x3=layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
    x3=layers.Conv2D(filters=48,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x3)
    x3=layers.Conv2D(filters=64,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x3)

    x4=tf.keras.layers.Concatenate()([x,x1,x2,x3])
    x4=layers.Conv2D(filters=384,kernel_size=(1,1),strides=(1,1),padding='same')(x4)
    
    x5=layers.Add()([x,x4]) 
    x5=layers.activation.ReLU()(x5)
    return x5



def Inception_ResNet_Block_B(x):
    x=layers.activation.ReLU()(x)

    x1=layers.Conv2D(filters=192,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)

    x2=layers.Conv2D(filters=128,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x2=layers.Conv2D(filters=160,kernel_size=(1,7),strides=(1,1),padding='same',activation='relu')(x2)
    x2=layers.Conv2D(filters=192,kernel_size=(7,1),strides=(1,1),padding='same',activation='relu')(x2)
    
    x3=tf.keras.layers.Concatenate()([x1,x2])

    x3=layers.Conv2D(filters=1154,kernel_size=(1,1),strides=(1,1),padding='same')(x3)
    
    x4=layers.Add()([x,x3])

    x4=layers.activation.ReLU()(x4)

    return x4


def Inception_ResNet_Block_C(x):
    x=layers.activation.ReLU()(x)

    x1=layers.Conv2D(filters=192,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)

    x2=layers.Conv2D(filters=192,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x2=layers.Conv2D(filters=224,kernel_size=(1,3),strides=(1,1),padding='same',activation='relu')(x2)
    x2=layers.Conv2D(filters=256,kernel_size=(3,1),strides=(1,1),padding='same',activation='relu')(x2)
    
    x3=tf.keras.layers.Concatenate()([x1,x2])

    x3=layers.Conv2D(filters=2048,kernel_size=(1,1),strides=(1,1),padding='same')(x3)
    
    x4=layers.Add()([x,x3])

    x4=layers.activation.ReLU()(x4)

    return x4


def Reduction_Block_A(input_1,input_2,input_3,input_4,input_5):
    x=tf.keras.layers.Concatenate()([input_1,input_2,input_3,input_4,input_5]) 
    x=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)

    x1=layers.Conv2D(filters=384,kernel_size=(3,3),strides=(2,2),padding='same',activation='relu')(x)

    x2=layers.Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x2=layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x2)
    x2=layers.Conv2D(filters=384,kernel_size=(3,3),strides=(2,2),padding='same',activation='relu')(x2)

    x3=tf.keras.layers.Concatenate()([x1,x2])
    
    return x3 



def Reduction_Block_B(x):
    x1=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)

    x2=layers.Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x2=layers.Conv2D(filters=384,kernel_size=(3,3),strides=(2,2),padding='same',activation='relu')(x2)
    

    x3=layers.Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x3=layers.Conv2D(filters=288,kernel_size=(3,3),strides=(2,2),padding='same',activation='relu')(x3)

    x4=layers.Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x4=layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
    x4=layers.Conv2D(filters=320,kernel_size=(3,3),strides=(2,2),padding='same',activation='relu')(x)
    

    x5=tf.keras.layers.Concatenate()([x1,x2,x2,x3,x4])
    
    return x5 

In [8]:
def Inception_ResNet():
    input=layers.Input(shape=(299,299,3))
    x=Stem(input)
    
    x1=Inception_ResNet_Block_A(x)
    x2=Inception_ResNet_Block_A(x)
    x3=Inception_ResNet_Block_A(x)
    x4=Inception_ResNet_Block_A(x)
    x5=Inception_ResNet_Block_A(x)
    
    x6=Reduction_Block_A(x1,x2,x3,x4,x5)
    for i in range(10):
        x6=Inception_ResNet_Block_B(x6)
    x6=Reduction_Block_B(x6)
    for i in range(5):
        x6=Inception_ResNet_Block_C(x6)
    
    x6=layers.GlobalAveragePooling2D()(x6)
    x6=layers.Dropout(0.8)
    x6=layers.Dense(units=10,activation='softmax')

    return x6
    
model=Inception_ResNet()
model = tf.keras.models.Model(inputs,[output,ax1,ax2],name = 'GoogleNet')

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 75, 75, 64), (None, 150, 150, 96)]